# Read the data

In [90]:
import os
import pandas as pd

from fastapi import FastAPI
from typing import List
from models import CountryData
import re

In [27]:
filepath = "../data/TPI ASCOR data - 13012025/ASCOR_assessments_results.xlsx" # Specify the correct path to the file
df_assessments = pd.read_excel(filepath)

# Convert the date columns to datetime type so we can filter by year later
df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])
df_assessments['Publication date'] = pd.to_datetime(df_assessments['Publication date'])

/var/folders/kv/l9vcg1gs7xb1tfhh37wfc4_w0000gn/T/ipykernel_31605/3951065428.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])


# Define the function

In [95]:
country = 'Italy'
assessment_year = 2024

data = df_assessments[(df_assessments["Country"] == country) & (df_assessments["Assessment date"].dt.year == assessment_year)]
data = data[[col for col in data.columns if col.startswith(("area", "indicator", "metric"))]]

remap_column_names = {col: re.sub(".*?\s", "", col) for col in data.columns}
data = data.rename(columns=remap_column_names)

data.iloc[0]

EP.1                         Partial
EP.1.a                           Yes
EP.1.b                            No
EP.1.c                            No
EP.2                         Partial
                      ...           
CF.4                             NaN
CF.4.i       2.24 MW/US$ billion GDP
CF.4.ii     20.56 MW/US$ billion GDP
CF.4.iii     0.00 MW/US$ billion GDP
CF.4.iv      0.12 MW/US$ billion GDP
Name: 55, Length: 74, dtype: object

In [ ]:

    #get areas. an EP is composed of dictionaries of EP1, EP2, EP3
    EP = {EP_i for EP_i in pillars_i_dict if EP_i.startswith("EP")}
    CP = {CP_i for CP_i in pillars_i_dict if CP_i.startswith("CP")}
    CF = {CF_i for CF_i in pillars_i_dict if CF_i.startswith("CF")}

In [28]:
def get_country_data(country: str, assessment_year: int):
    data = df_assessments[(df_assessments["Country"] == country) & (df_assessments["Assessment date"].dt.year == assessment_year)]
    data = data[[col for col in data.columns if col.startswith(("area", "indicator", "metric"))]]

    #rename columns so they start with the pillars
    remap_column_names = {col: re.sub(".*?\s", "", col) for col in data.columns}
    data = data.rename(columns=remap_column_names)

    #get flat Pandas series of country data
    data = data.iloc[0]
    data = data.fillna("")


    #get pillar
    pillars = [{'name': pillar, 'areas': "notdoneyet"} for pillar in ['EP', 'CP', 'CF']]


    area_columns = [col for col in data.index if col.startswith("area")]
    data = data[area_columns]
    data['country'] = country
    data['assessment_year'] = assessment_year

    output_dict = {'pillars': {pillar for pillar in pillars}}

    output = CountryData(**output_dict)

    return output

In [68]:
unique_values = [df_assessments[col].unique().tolist() for col in df_assessments.columns if col.startswith("country")]

In [70]:
unique_values = [df_assessments['Country'].unique().tolist()]

In [81]:
countries = list((set(sum(unique_values, []))))

# Test how the function behaves

In [21]:
get_country_data("Italy", 2024)

CountryData(country='Italy', assessment_year=2024, EP_1='Partial', EP_2='Partial', EP_3='Partial', CP_1='No', CP_2='Partial', CP_3='Partial', CP_4='Partial', CP_5='Partial', CP_6='Partial', CF_1='No', CF_2='Exempt', CF_3='Partial', CF_4='')

# Testing out Pydantic Models

In [73]:
from pydantic import BaseModel, Field
from typing import Literal